In [48]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 10 # vehicles

In [51]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [52]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

11-07-23 23:01:42-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
11-07-23 23:01:42-INFO-creating combinations
11-07-23 23:01:42-INFO-9900	 nR*(nR-1)
11-07-23 23:01:42-INFO-Reduction of feasible pairs by 99.85%
11-07-23 23:01:42-INFO-Degree 2 	Completed
11-07-23 23:01:42-INFO-trips to extend at degree 2 : 43
11-07-23 23:01:43-INFO-At degree 2 feasible extensions found out of 1 searched
11-07-23 23:01:43-INFO-Degree 3 	Completed
11-07-23 23:01:43-INFO-trips to extend at degree 3 : 1
11-07-23 23:01:43-INFO-At degree 3 feasible extensions found out of 0 searched
11-07-23 23:01:43-INFO-Degree 4 	Completed
11-07-23 23:01:43-INFO-Max degree reached 4
11-07-23 23:01:43-INFO-Trips still possible to extend at degree 4 : 0


In [53]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

11-07-23 23:01:43-INFO-Matching 100 trips to 144 rides in order to min u_veh
11-07-23 23:01:43-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,275 
reduced by matching to:               24,424


In [54]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],1.575500,85,1,[1.5755],"[0, 85]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 484705811, 44832713]","[None, 0, 0]",node time req_id od 0 N...,85,0.85,1.278000,0.255600,1.0224
1,[1],3.094000,167,1,[3.0940000000000003],"[249, 167]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44772101, 1552651408]","[None, 1, 1]",node time req_id od 0 ...,167,1.67,2.509500,0.501900,2.0076
2,[2],4.302500,232,1,[4.3025],"[377, 232]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44845277, 44786364]","[None, 2, 2]",node time req_id od 0 NaN...,232,2.32,3.490500,0.698100,2.7924
3,[3],6.820000,368,1,[6.82],"[410, 368]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1436427125, 3216669367]","[None, 3, 3]",node time req_id od 0 ...,368,3.68,5.532000,1.106400,4.4256
4,[4],5.563500,300,1,[5.5634999999999994],"[440, 300]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 3669218091, 44808578]","[None, 4, 4]",node time req_id od 0 ...,300,3.00,4.513500,0.902700,3.6108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,"[74, 77]",12.862200,463,21,"[7.654875, 5.207325]","[10495.5, 71, 292, 100]","[74, 77]","[77, 74]",2,139,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 672526443, 2381861250, 44817443, 484705...","[None, 74, 77, 77, 74]",node time req_id od 0 ...,463,4.63,8.248500,1.649700,6.5988
140,"[74, 78]",11.986125,668,21,"[8.030249999999999, 3.955875]","[10359.0, 356, 248, 64]","[74, 78]","[78, 74]",2,140,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 672526443, 44779743, 1391417274, 484705...","[None, 74, 78, 78, 74]",node time req_id od 0 ...,668,6.68,7.754625,1.550925,6.2037
141,"[75, 78]",10.643825,473,21,"[6.5082249999999995, 4.1356]","[10551.5, 110, 248, 115]","[75, 78]","[78, 75]",2,141,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1510592129, 44779743, 1391417274, 44866...","[None, 75, 78, 78, 75]",node time req_id od 0 ...,473,4.73,6.940125,1.388025,5.5521
142,"[79, 77]",14.551800,596,21,"[9.5515, 5.0003]","[10709.0, 71, 292, 233]","[79, 77]","[77, 79]",2,142,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44713931, 2381861250, 44817443, 1803176...","[None, 79, 77, 77, 79]",node time req_id od 0 ...,596,5.96,9.747000,1.949400,7.7976


In [63]:
responses = []
avg_kpi = []
idle_time = []

for i in range(2, 3):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].kpi_veh_pricing.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].kpi_veh_pricing.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

0.5
12-07-23 10:00:44-CRITICAL-this is request 0 with [0] available rides.
12-07-23 10:00:44-CRITICAL-ride 0 available [1]
12-07-23 10:00:44-CRITICAL-this is reuqest 0 with [0] still available rides.
12-07-23 10:00:44-CRITICAL-vehicle 1 has 1 choices
0.5
12-07-23 10:00:44-CRITICAL-this is request 1 with [1] available rides.
12-07-23 10:00:44-CRITICAL-ride 1 available [1]
12-07-23 10:00:44-CRITICAL-this is reuqest 1 with [1] still available rides.
12-07-23 10:00:44-CRITICAL-vehicle 10 has 1 choices
0.5
12-07-23 10:00:44-CRITICAL-this is request 2 with [2, 128] available rides.
12-07-23 10:00:44-CRITICAL-ride 2 available [1]
12-07-23 10:00:44-CRITICAL-ride 128 available [1, 0]
12-07-23 10:00:44-CRITICAL-this is reuqest 2 with [2, 128] still available rides.
12-07-23 10:00:44-CRITICAL-vehicle 7 has 2 choices
0.5
12-07-23 10:00:44-CRITICAL-this is request 3 with [3] available rides.
12-07-23 10:00:44-CRITICAL-ride 3 available [1]
12-07-23 10:00:44-CRITICAL-this is reuqest 3 with [3] still 

In [55]:
params.kpi=1

In [56]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

11-07-23 23:01:54-INFO-Matching 100 trips to 144 rides in order to min u_veh
11-07-23 23:01:54-INFO-Problem solution: Optimal. 
Total costs for single trips:         26,275 
reduced by matching to:               24,424
0.5
11-07-23 23:01:57-CRITICAL-this is request 0 with [0] available rides.
11-07-23 23:01:57-CRITICAL-ride 0 available [1]
11-07-23 23:01:57-CRITICAL-this is reuqest 0 with [0] still available rides.
11-07-23 23:01:57-CRITICAL-vehicle 1 has 1 choices
0.5
11-07-23 23:01:57-CRITICAL-this is request 1 with [1] available rides.
11-07-23 23:01:57-CRITICAL-ride 1 available [1]
11-07-23 23:01:57-CRITICAL-this is reuqest 1 with [1] still available rides.
11-07-23 23:01:57-CRITICAL-vehicle 8 has 1 choices
0.5
11-07-23 23:01:57-CRITICAL-this is request 2 with [2, 128] available rides.
11-07-23 23:01:57-CRITICAL-ride 2 available [1]
11-07-23 23:01:57-CRITICAL-ride 128 available [1, 0]
11-07-23 23:01:57-CRITICAL-this is reuqest 2 with [2, 128] still available rides.
11-07-23 23:01:5

In [57]:
pd.DataFrame(sim.vehs[1].myrides).head(50)

,veh,pos,t,event,paxes
0,1,4.484201e+07,0,STARTS_DAY,[]
1,1,4.484201e+07,0,OPENS_APP,[]
2,1,4.484201e+07,0,RECEIVES_REQUEST,[]
3,1,4.484201e+07,15,ACCEPTS_REQUEST,[]
4,1,4.484201e+07,35,IS_ACCEPTED_BY_TRAVELLER,[None]
5,1,4.847058e+08,53,ARRIVES_AT_PICKUP,[]
6,1,4.847058e+08,53,MEETS_TRAVELLER_AT_PICKUP,[]
7,1,4.847058e+08,83,DEPARTS_FROM_PICKUP,[0]
8,1,4.483271e+07,168,ARRIVES_AT_DROPOFF,[0]
9,1,4.483271e+07,377,RECEIVES_REQUEST,[]


In [58]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,12,0.0,False,0,0,9386.0,150.0,0,279.0,0,...,0,0,0,8382.0,1892.0,0,1150.0,789.0,4551.0,22.378
2,9,0.0,False,0,0,9910.0,105.0,0,238.0,0,...,0,0,0,7421.0,2145.0,0,1510.0,613.0,3153.0,26.383
3,10,0.0,False,0,0,10343.0,105.0,0,288.0,0,...,0,0,0,8137.0,1525.0,0,901.0,693.0,5018.0,21.681
4,11,0.0,False,0,0,9798.0,123.0,0,218.0,0,...,0,0,0,7521.0,2307.0,0,1302.0,671.0,3241.0,26.890
5,9,0.0,False,0,0,9463.0,135.0,0,180.0,0,...,0,0,0,7881.0,2521.0,0,1149.0,585.0,3626.0,25.251
6,14,0.0,False,0,0,6375.0,165.0,0,381.0,0,...,0,0,0,8956.0,3549.0,0,1753.0,966.0,2688.0,42.119
7,8,0.0,False,0,0,10053.0,120.0,0,160.0,0,...,0,0,0,8055.0,2041.0,0,930.0,520.0,4564.0,20.440
8,11,0.0,False,0,0,6053.0,120.0,0,265.0,0,...,0,0,0,10081.0,2932.0,0,1818.0,715.0,4616.0,38.326
9,13,0.0,False,0,0,9416.0,165.0,0,302.0,0,...,0,0,0,7257.0,2733.0,0,1336.0,857.0,2331.0,31.535
10,3,0.0,False,0,0,10410.0,45.0,0,60.0,0,...,0,0,0,9810.0,817.0,0,367.0,195.0,8431.0,8.189


In [59]:
sim.vehs[1].rdf

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
0,[0],1.575500,85,1,[1.5755],"[0, 85]",[0],[0],1,0,...,node time req_id od 0 N...,85,0.85,1.278000,0.255600,1.0224,384,384.85,192.425,-191.4026
2,[2],4.302500,232,1,[4.3025],"[377, 232]",[2],[2],1,2,...,node time req_id od 0 NaN...,232,2.32,3.490500,0.698100,2.7924,1852,1854.32,927.160,-924.3676
13,[13],3.003000,162,1,[3.003],"[1123, 162]",[13],[13],1,13,...,node time req_id od 0 ...,162,1.62,2.436000,0.487200,1.9488,932,933.62,466.810,-464.8612
19,[19],4.634000,250,1,[4.634],"[1872, 250]",[19],[19],1,19,...,node time req_id od 0 ...,250,2.50,3.759000,0.751800,3.0072,187,189.50,94.750,-91.7428
23,[23],8.614500,465,1,[8.6145],"[2624, 465]",[23],[23],1,23,...,node time req_id od 0 ...,465,4.65,6.987000,1.397400,5.5896,2608,2612.65,1306.325,-1300.7354
34,[34],1.096000,59,1,[1.096],"[3465, 59]",[34],[34],1,34,...,node time req_id od 0 ...,59,0.59,0.889500,0.177900,0.7116,1071,1071.59,535.795,-535.0834
43,[43],4.205500,227,1,[4.2055],"[4643, 227]",[43],[43],1,43,...,node time req_id od 0 ...,227,2.27,3.411000,0.682200,2.7288,1110,1112.27,556.135,-553.4062
51,[51],0.818500,44,1,[0.8185],"[5679, 44]",[51],[51],1,51,...,node time req_id od 0 ...,44,0.44,0.664500,0.132900,0.5316,1889,1889.44,944.720,-944.1884
113,"[67, 68]",5.573725,292,20,"[2.7215249999999997, 2.8522]","[8936.5, 115, 81, 96]","[67, 68]","[67, 68]",2,113,...,node time req_id od 0 NaN...,292,2.92,3.790125,0.758025,3.0321,788,790.92,395.460,-392.4279
91,[91],4.865500,263,1,[4.8655],"[12866, 263]",[91],[91],1,91,...,node time req_id od 0 ...,263,2.63,3.945000,0.789000,3.1560,430,432.63,216.315,-213.1590


In [62]:
pd.DataFrame(sim.vehs[1].myrides)

,veh,pos,t,event,paxes
0,1,4.484201e+07,0,STARTS_DAY,[]
1,1,4.484201e+07,0,OPENS_APP,[]
2,1,4.484201e+07,0,RECEIVES_REQUEST,[]
3,1,4.484201e+07,15,ACCEPTS_REQUEST,[]
4,1,4.484201e+07,35,IS_ACCEPTED_BY_TRAVELLER,[None]
...,...,...,...,...,...
78,1,5.124502e+09,12924,ARRIVES_AT_PICKUP,[]
79,1,5.124502e+09,12924,MEETS_TRAVELLER_AT_PICKUP,[]
80,1,5.124502e+09,12954,DEPARTS_FROM_PICKUP,[91]
81,1,1.679761e+09,13217,ARRIVES_AT_DROPOFF,[91]
